In [1]:
using SparseArrays, LinearAlgebra, DelimitedFiles, IndexedGraphs, Plots
using IntervalUnionArithmetic
using ProgressMeter
#using Pkg
#Pkg.activate("../")
using Causality
using Graphs
using Dictionaries

# Generation of Epidemics (REAL Graph)

In [ ]:
#=DS=readdlm("./RealGraph/High-School_data_2013.csv");
DS[:,1] .-= minimum(DS[:,1])
timescale = maximum(DS[:,1])/20
DS[:,1] ./= timescale=#


DS=readdlm("./RealGraph/detailed_list_of_contacts_Hospital.dat_");
timescale = 20000 
DS[:,1] ./= timescale

#DS=readdlm("./tij_SFHH.dat_");
#timescale = 7000;

In [6]:
nodes = unique(DS[:,[2,3]])
T = Float64(maximum(DS[:,1]))
N = length(nodes)
G = SimpleGraph(N)
id = Dictionary(nodes, [i for i = 1:N]); 
for st = 1:size(DS,1)
    add_edge!(G,id[DS[st,2]],id[DS[st,3]]) 
end
jobs = Dictionary([i for i = 1:N],(unique([unique(DS[:,[2,4]], dims=1) ; unique(DS[:,[3,5]], dims=1)],dims=1))[:,2])
G = IndexedBiDiGraph(G)

{75, 2278} directed sparse Int64 graph
⣮⡻⠾⡇⠯⣽⠿⢿⣿⢭⠻⢿⠀⠹⠿⠿⠿⠌⠿⢿⠠⠨⠹⠛⠏⡿⠏⠇⠀⠐⠘⠬⢬⠔⠠⠷⠋⠅
⠾⠧⡮⡫⡿⣿⣿⠾⠾⣾⡤⢴⠐⡲⢿⠷⠶⢾⠿⠿⠶⠀⡶⢿⠜⠴⡷⠆⠆⠲⠤⠲⠴⡇⡢⠶⠆⠄
⣏⣧⣿⣯⣎⡹⣿⡿⢭⣯⣹⣽⠀⣼⣽⣭⣭⠭⣍⣯⣉⣈⣙⣸⠠⣹⣿⣃⣄⣘⠉⡱⢋⣥⡌⢟⡁⠄
⣿⣇⣻⡟⣿⡿⣮⡻⡴⣗⢇⣞⠄⢸⣿⡇⣄⠚⣞⣿⣶⣗⣿⣨⣈⢼⡛⠇⡍⢌⠀⢠⢑⡆⠀⡿⠇⡄
⡟⣟⣺⣧⡧⣷⢴⢯⢮⡳⣾⢳⠀⣴⣳⡗⢺⢄⣻⣴⣲⣗⣲⢂⢡⣯⡅⠄⠤⢠⠑⣤⢴⣤⠐⠀⠤⠄
⣿⣆⢀⣏⣗⣾⣩⢵⢾⣛⢀⡰⣀⢰⣲⣆⣴⢄⣼⡓⠩⠎⠩⠀⡆⢶⢖⣅⢉⠸⠀⢑⠪⡀⠀⢜⠅⠀
⣄⡀⢰⡠⣀⣤⣀⣁⢀⣤⢀⣘⠮⠃⣤⣄⣀⣀⣀⢀⣛⣘⣛⣸⣀⣀⣃⡀⡀⣀⠀⣀⣘⣓⡓⡀⡀⠀
⣿⡇⢿⡗⡗⣿⠿⠿⢽⠾⠸⢾⠀⢿⣮⡻⡿⠝⡿⢿⣿⡍⣽⣿⠾⢿⠷⡆⠂⠀⢈⠰⠾⠟⠯⡦⠶⠂
⡛⠇⣸⣇⡇⡟⣠⠙⠚⢖⠐⢟⠀⢸⣟⠏⡊⠈⡗⠈⠹⠋⠉⠛⠒⢚⠒⡀⠀⠐⠐⠚⠀⠁⠀⢁⠐⠀
⣿⣇⣿⡇⡧⣽⣾⣽⢛⣾⢶⠻⠀⢘⣿⣏⡙⠉⣮⡻⣜⣩⣭⢯⢚⢚⡷⣆⣆⣠⠂⣆⢬⡄⠀⢦⢢⠀
⡀⡂⠘⠃⡃⢸⢼⢿⢼⢾⡣⠆⣛⢸⡟⠿⡷⠂⡖⣹⠊⠀⠀⠀⠀⢸⠀⠠⢀⢀⠀⠀⠀⠀⠀⠀⠀⠀
⣷⠂⣼⣏⣓⣸⡛⣻⠸⢚⠃⠂⣛⣸⣷⣿⣧⠀⡧⣟⠀⠀⠀⠀⡀⢺⠀⣼⢸⠀⠀⢠⠀⠀⠀⡀⠀⠂
⣯⡥⢒⡅⣄⣢⣂⣜⡥⣶⢨⣍⠀⢸⣾⣇⣸⢀⣺⢐⣀⣀⣠⣈⠊⠀⣎⡒⠐⣀⡀⣹⡀⠀⠀⣩⣈⠀
⠯⠅⠹⠏⠿⢻⠿⠌⠁⠍⠜⢵⠉⠸⠹⠧⠘⠠⠹⢯⠀⡀⣀⣤⢪⠹⣤⡻⣸⠀⠀⠘⠀⠀⠀⡄⡀⠀
⢀⠀⢨⡁⣀⢹⡃⢍⠀⣃⣃⡐⠀⢨⠈⠀⢀⠀⠈⣹⠀⢐⠒⠒⠐⢠⠒⠚⢊⠐⠄⠢⠂⠀⠀⠂⢐⠀
⡒⡄⢠⡃⢇⡠⠀⣀⠑⣤⢄⢀⠀⢠⢂⡐⣰⠀⠨⢤⠀⠀⠀⣀⣄⣨⣀⠀⠠⡁⠀⠀⡀⠀⠀⡐⠀⡀
⢂⠗⠴⠧⠏⣴⠱⠴⠐⣷⠊⠢⢶⢸⣾⠇⠄⠀⠂⠷⠀⠀⠀⠀⠀⠈⠀⠀⠈⠀⠀⠈⠀⠀⠀⠀⠀⠀
⢤⡆⢨⡎⣦⢍⣤⡤⠐⠀⣀⢄⠙⠨⠫⡧⠄⢀⠠⣄⠀⠀⠀⠠⡄⣠⠀⠤⠠⠀⢀⠠⠀⠀⠀⠀⠀⠀
⠏⠄⠈⠅⠁⠌⠉⠥⠀⠇⠁⠁⠀⠈⠸⠃⠐⠀⠈⠒⠀⠀⠠⠀⠂⠘⠀⠈⠐⠐⠀⠠⠀⠀⠀⠀⠀⠀

In [7]:
H = sparse(G)
contacts = Dictionary([(i,j) for i=1:N for j=1:N if H[i,j] != 0 ] ,[i for i = 1:nnz(G.A)])
meet_time=20/timescale
masks = [intervalUnion(∅) for g=1:nnz(G.A)]
for st = 1:size(DS,1)
    masks[contacts[(id[DS[st,2]],id[DS[st,3]])]] = masks[contacts[(id[DS[st,2]],id[DS[st,3]])]] ∪ intervalUnion(DS[st,1]-meet_time,DS[st,1])
    #symmetrization
    masks[contacts[(id[DS[st,3]],id[DS[st,2]])]] = masks[contacts[(id[DS[st,3]],id[DS[st,2]])]] ∪ intervalUnion(DS[st,1]-meet_time,DS[st,1])
end
V = [MaskedRate(UnitRate(),mask) for mask in masks];

# Generation of Epidemics (SIMULATED Graph)

In [2]:
T, N = 20.0, 1000
timescale = 1
#G = Causality.makeBarabasi(N,k=2)
#ρ = 3/N
#G = Causality.makeGNP(N,ρ)
G = Causality.makeProximity(N,2/N)
G.A 


1000×1000 SparseMatrixCSC{IndexedGraphs.NullNumber, Int64} with 6262 stored entries:
⢔⡵⣬⣥⡯⢱⢖⡩⡗⠲⡦⣻⢨⢴⡕⣒⡪⣴⠵⡡⡧⣞⠶⣇⡳⣯⣫⠵⣷⠙⡻⡓⢪⣝⡷⢼⣿⣆⣾⡅
⠆⣿⡥⡫⢶⠟⡝⣺⢾⡉⣩⡗⡒⣒⢲⢳⡿⡵⡲⡕⡷⣊⠋⣆⡞⠫⠽⠆⢴⣾⢵⡦⠸⣥⡏⢣⣏⡶⣯⡄
⢏⣋⣼⠗⢎⣱⢋⣷⣼⢷⣽⣓⡚⣾⣻⣾⡬⢌⡿⣓⣗⢳⣖⢙⢣⣵⠮⡯⣿⠿⡱⡝⠽⢎⠯⣸⣝⡷⠬⢝
⡜⡱⣳⣩⢯⣴⡠⡮⣞⠱⢣⣝⢲⡝⠭⡰⣏⣛⢽⢷⣫⣋⢺⡪⠷⢥⡓⣼⣜⠢⢷⣟⣗⣿⢹⣺⣗⢹⡱⣿
⢹⡉⡞⠳⢶⣟⢞⡙⣤⣻⠖⡾⠑⢎⣗⡽⢻⠟⣶⡗⡏⢾⣗⢟⣍⢍⡵⣫⡼⢒⡟⡓⢉⢰⢽⠒⢿⠾⡯⢷
⣬⣫⢧⠾⢷⢻⣍⢶⣸⡥⠊⣤⣚⡓⣞⣎⣇⢉⡾⣊⣊⣿⣁⡿⡩⡅⢷⠛⣾⠛⢽⣽⣃⢸⣹⣹⣵⡺⢧⠭
⢂⣖⢸⢨⣺⣬⣜⠶⡱⢄⢾⠸⣡⣾⣙⣉⠞⢸⣵⢽⣯⣳⢫⢲⣯⠫⠴⣜⡿⢋⡞⣩⢞⠶⠵⣞⢴⠶⡕⢧
⢱⢩⢼⣒⣻⣾⢃⡣⣝⡽⡺⢽⡗⢸⢕⡱⠯⢆⡷⡛⣻⡎⡯⢑⡥⠾⣞⠭⢾⡺⢼⠞⠸⢘⣾⡕⣸⡘⣟⠊
⢊⣮⢟⡯⡂⢏⣯⢹⣿⠖⡍⢙⣚⣁⠫⢇⢮⠗⢟⠿⢵⢮⢚⡭⢻⣺⣰⡷⣽⢻⣟⣪⢛⣾⣷⢭⣶⣋⡓⢗
⠕⡣⢜⠮⢿⢫⢷⣗⢼⠿⡺⢫⣕⣟⣽⠫⣿⡕⡔⠉⣯⣱⣻⡋⣇⣹⢙⢵⢿⢶⠵⢿⠯⢬⢏⣿⣮⢏⢇⡟
⣩⢯⡹⢫⢽⣙⡯⢺⣫⣍⣮⣼⢯⣻⡻⠾⡱⣗⢏⣻⠮⡣⢪⣱⣶⢕⡿⣑⣨⠭⣱⡓⢟⣾⣕⣷⢍⡿⡪⡗
⠼⢧⠫⢤⣜⢙⡺⡲⣽⢝⣥⡼⢫⣒⢏⢋⡞⡴⡿⠺⢎⣲⠮⣧⣯⡿⠛⢸⢞⢒⡿⢿⠤⣲⠋⣺⡆⡴⢮⠶
⡽⣮⡾⡉⢍⣶⠝⣇⡇⢝⠇⠮⡯⡛⣡⡏⣻⣲⣍⣹⢜⢟⣯⡿⣚⢘⣥⠯⣽⡨⣗⡋⣶⣻⠥⠛⠎⢫⣫⡆
⢏⡞⠳⠇⡮⡧⣙⣬⡵⣫⣽⠓⣐⢧⡞⡝⢴⡾⢗⣔⢟⢫⣛⣀⡥⡟⠚⡤⣭⢥⢸⡇⡯⠛⠩⣫⣜⢟⡴⠆
⣝⠛⣰⣷⣿⡟⠲⡙⢲⢋⣾⠛⡿⢋⣺⡳⣷⣛⢻⣗⡆⡞⢺⢑⡓⡻⠇⣟⣞⡽⣩⣗⢻⢴⢟⢸⣟⡧⣗⠃
⢿⠪⠱⡷⣕⠮⣽⢷⢿⠩⣗⣷⡞⣩⣲⠗⡻⣹⣵⣇⢵⠺⣿⣏⡽⠹⠶⠶⢧⢾⡴⠏⠾⢓⢇⣼⣩⣥⣻⢠
⣎⢶⠖⣦⡳⢇⣽⣽⢃⣐⣉⣘⢺⡕⣒⢂⣻⣴⡋⣇⣻⣵⢠⣣⣼⣻⣯⠋⢛⣖⢾⢃⠿⡧⢸⢍⡓⣷⣚⣗
⣙⣏⠯⣉⣋⣣⣳⣲⢳⠓⣗⣺⣱⢧⢞⠿⡝⣟⣯⣵⢵⣽⣫⣠⣥⠃⡧⣢⣛⣑⣉⣵⡖⢖⢊⠔⣧⣾⣵⠕
⠻⢿⢫⡽⢷⡽⣝⣙⣻⡗⣱⡻⢰⡗⣒⠺⡼⢻⡮⢟⣧⡵⢈⡭⡮⣁⣶⢝⠿⡽⠇⣾⢽⣬⣩⣿⠤⠃⢿⣯
⠞⠿⠋⠿⣆⢇⣵⣮⢯⣏⡍⡗⠵⣍⡻⠙⢽⢌⣭⠵⢮⠮⢪⡗⠫⠾⠰⠏⠽⠙⠛⣚⢾⢼⢕⠟⡿⣷⠠⡢

In [3]:
V = fill(MaskedRate(UnitRate(),intervalUnion(0,T)), ne(G));

In [4]:
getpar(pseed,autoinf::GaussianRate,inf_in::GaussianRate) = 
    [fill(pseed, 1, N);
     fill(autoinf.a, 1, N); fill(autoinf.b, 1, N); fill(autoinf.c, 1, N);
     fill(inf_in.a,   1, N); fill(inf_in.b, 1, N); fill(inf_in.c, 1, N);
    ]

getpargen(pseed, autoinf::GaussianRate, inf_out::GaussianRate) = 
    [pseed autoinf.a autoinf.b autoinf.c inf_out.a inf_out.b inf_out.c]


getpargen (generic function with 1 method)

In [5]:
##### Initialize generation parameters
ε = 1e-10

λ = 0.4
pseed = 1/N
autoinf = GaussianRate(ε, T/2, 1/ε)
inf_in = GaussianRate(1.0, T/2, 1/ε)
inf_out = GaussianRate(λ, T/2, 1/ε)



θp = getpar(pseed, autoinf, inf_in);
θpgen = getpargen(pseed, autoinf, inf_out);

In [6]:
#Initialize min/max boundaies
pseed_min = ε
pseed_max = 1-ε
rate_min = GaussianRate(ε  ,  -10*T,   ε)
rate_max = GaussianRate(1/ε  ,  10*T,   1/ε)

θmin = getpar(pseed_min , rate_min, rate_min);
θmax = getpar(pseed_max , rate_max, rate_max);

θgenmin = getpargen(pseed_min, rate_min, rate_min);
θgenmax = getpargen(pseed_max, rate_max, rate_max);

In [7]:
const Igen = GenerativeSI{GaussianRate,GaussianRate} 
const Igauss = GaussianInferentialSI

GaussianInferentialSI

In [15]:
Mp = StochasticModel(Igen, T, θp, G, θpgen,V);
sample! = Sampler(Mp);
xtrue = zeros(N)
sample!(xtrue)
println(sum(xtrue .< T)/N)
#scatter(xtrue)

0.567


In [16]:
function Cau_infereSI(G, V, N, T, pseed, autoinf, inf_out, O; ε=1e-10)
    pseed_min = ε
    pseed_max = 1-ε
    rate_min = GaussianRate(ε  ,  -10*T,   ε)
    rate_max = GaussianRate(1/ε  ,  10*T,   1/ε)

    θmin = getpar(pseed_min , rate_min, rate_min);
    θmax = getpar(pseed_max , rate_max, rate_max);

    θgenmin = getpargen(pseed_min, rate_min, rate_min);
    θgenmax = getpargen(pseed_max, rate_max, rate_max);
    
    
    θp2gen = getpargen(pseed, autoinf, inf_out);
    inf_in = GaussianRate(1., T/2, 3*T );
    θp2 = getpar(pseed, autoinf, inf_in);
    Mp2 = StochasticModel(Igen, T, θp2, G, θp2gen, V);
    θ = getpar(pseed, autoinf, inf_in);
    M = StochasticModel(Igauss, T, θ, G, θp2gen, V);    
    
    ProgressMeter.ijulia_behavior(:clear)
    F = descend!(Mp2, O; M=M, numsamples=1000, numiters=100, 
         θmin=θmin, θmax=θmax,θgenmin=θgenmin, θgenmax=θgenmax, descender=SignDescender(0.1),
         hyperdescender=SignDescender(0.));
    F = descend!(Mp2, O; M=M, numsamples=1000, numiters=100, 
         θmin=θmin, θmax=θmax,θgenmin=θgenmin, θgenmax=θgenmax, descender=SignDescender(0.03),
         hyperdescender=SignDescender(0.));
    return prior(M, numsamples=10000);
end

function marginal(i, t, stats)
    numsamp = size(stats,1)
    sum(stats[:,i] .< t)/numsamp
end

function reweighted_marginal(i, t, stats, weights)
    numsamp = size(stats,1)
    @assert numsamp == size(weights,1)
    weights ./= maximum(weights)
    sum(weights .* (stats[:,i] .< t))/sum(weights)
end


function tpr(xtrue, rank)
    if sum(xtrue) == 0
        return ones(N)
    end
    return cumsum(xtrue[rank]) ./ sum(xtrue)
end

function fpr(xtrue, rank) 
    N = size(rank,1)
    if sum(xtrue) == N
       a = zeros(N) 
       a[N] = 1
       return a 
    end   
    return (range(1,N,length=N) .- cumsum(xtrue[rank])) ./ (range(1,N,length=N) .- cumsum(xtrue[rank]) )[end]
end

function ROC(xtrue, p)
    N = size(xtrue,1)
    rank = sortperm(p, rev=true)
    return fpr(xtrue, rank) , tpr(xtrue, rank)
end

function AUROC(ROC)
    N = size(ROC[1],1) 
    AU = 0
    for t = 1:N-1
        AU += ROC[2][t] * (ROC[1][t+1] - ROC[1][t])
    end
    return AU
end

using PyCall
@pyimport sib
function NonMarkovDynamicSibyl(N, T_cont, Λ, V, a, b, c, O, γ, f_rate; n_steps=20, maxit = 400, tol = 1e-14)
    dt = T_cont / n_steps
    T = Int(round(T_cont / dt))
    i = findnz(G.A)[1]
    j = findnz(G.A)[2]    
    contacts = [(i[a]-1, j[a]-1, t, 1.) for t in 1:T for a = 1:nnz(Λ.A) if (t * dt) in V[a].mask];
    obs = [[(i,-1,t) for t=1:T for i=0:N-1];
           [(i-1,s,Int(round(t/dt))) for (i,s,t,p) in O]]
    sort!(obs, lt=((i1,s1,t1),(i2,s2,t2))->(t1<t2))
    prob_sus = 0.5
    prob_seed=γ
    pseed = prob_seed / (2 - prob_seed)
    psus = prob_sus * (1 - pseed)
    g = sib.PiecewiseLinear(pycall(sib.RealParams, PyObject, [a*exp(-((t-b)/c)^2) * dt for t =1:T]))
    params = sib.Params(prob_r=sib.Exponential(mu=0), pseed=pseed, psus=psus,pautoinf=autoinf.a*dt,fp_rate=f_rate,fn_rate=f_rate, prob_i = g)
    f = sib.FactorGraph(contacts=contacts, observations=obs, params=params)
    sib.iterate(f, maxit=maxit,tol=tol)
    sib.iterate(f, maxit=maxit, damping=0.5, tol=tol)
    sib.iterate(f, maxit=maxit, damping=0.9, tol=tol)
    p_sib=[collect(n.bt) for n in f.nodes]
    m_sib = zeros(N, T)
    for i= 1:N
        m_sib[i,1] = p_sib[i][1] 
        for t=2:T
            m_sib[i,t] = m_sib[i,t-1] + p_sib[i][t]
        end
            end
    return m_sib
end

function MF_heu(N, T_cont, Λ, O, γ, λ ; dt=1/5, maxit = 400)
    T = Int(round(T_cont / dt))
    p_MF = zeros(N,T) #prob to be S
    p_MF[:,1] .= 1 - γ 
    obs_I = [o[1] for o in O if o[2] == 1]
    obs_S = [o[1] for o in O if o[2] == 0]
    for st = 1:maxit
        for t = 2:T
            for i = 1:N
                p_MF[i,t] = p_MF[i,t-1]
                for j in collect(Causality.outedges(Λ,1))
                    p_MF[i,t] *= 1 - λ*(1 - p_MF[j.dst,t-1])
                end
                if i in obs_I                    
                    o = O[i]
                    p_MF[i, Int(round((o[3]-5)/dt)) : end] .= 0
                end
                if i in obs_S
                    o = O[i]
                    p_MF[i, 1 : Int(round(o[3]/dt))] .= 1
                end
            end
        end
    end
    return 1 .- p_MF
end

MF_heu (generic function with 1 method)

In [ ]:
nobs_max, p = 300, 1e-5
Oav = []
nsteps_sib = 80
folderstring = "./SimulatedGraphs/SimulatedGraphData2/"
bins = 20
for trial = 1: 25
    G = Causality.makeProximity(N,2/N)
    V = fill(MaskedRate(UnitRate(),intervalUnion(0,T)), ne(G));
    Mp = StochasticModel(Igen, T, θp, G, θpgen,V);
    sample! = Sampler(Mp);
    sample!(xtrue)
    ni = sum(xtrue .< T) / N
    while !(0.4 < ni < 0.6)
        sample!(xtrue)
        ni = sum(xtrue .< T) / N
    end
    #O = [(i = rand(1:N) ; ti = T*(0.2+rand())/1.2; (i,xtrue[i] < ti,ti,p)) for l=1:nobs_max]
    O = [(ti = T*(1+rand())/2; (i,xtrue[i] < ti,ti,p)) for i=1:nobs_max]
    f_rate = 1e-5
    open("$(folderstring)try$(trial)xtrue.txt","w") do io
         writedlm(io,xtrue') 
    end
    Ofalse=[rand()< f_rate ? (o[1],!o[2],o[3],f_rate) : (o[1],o[2],o[3],f_rate) for o in O]
    for nobs in (10, 25, 40, 80, 160, 300)
        Oav = Ofalse[1:nobs] #available observations for the experiment
        statscau = Cau_infereSI(G, V, N, T, pseed, autoinf, inf_out, Oav)
        Osoft = [(o[1],o[2],o[3],max(0.05,f_rate)) for o in Oav]
        stats,weights_soft = softpostnoise(Mp, Osoft; numsamples=2 * 10^5);
        p_sib = NonMarkovDynamicSibyl(N, T, G, V, inf_out.a, inf_out.b, inf_out.c, Oav, 1/N, f_rate; n_steps=nsteps_sib, maxit = 40, tol = 1e-4);
        K = Causality.GaussMove(2.)
        xmc = zeros(N)
        sample!(xmc)
        Mmc = StochasticModel(Igen, T, θp, G, θpgen,V);
        stats_mh = Causality.metropolis_sampling_sequential(Mp, Oav, K; numsamples = 6 * 10^4,numsteps= 3, x = xmc); 

        struct HeuristicSI <: SI end
        θp2gen = getpargen(pseed, autoinf, inf_out);
        maskauto = fill(intervalUnion(0., T),N)
        maskinf = fill(intervalUnion(0., T),N)
        θfrench = getpar(pseed, autoinf, inf_in);
        Causality.individual(M::StochasticModel{HeuristicSI}, i::Int, θi = @view(M.θ[:,i]), θg = M.θgen ) = 
        @views IndividualSI(θi[1], 
            MaskedRate(GaussianRate(θi[2:4]...),maskauto[i]), 
            MaskedRate(UnitRate(),maskinf[i]), 
            GaussianRate(θg[5:7]...),)
        inf_start_time = T * ones(N)
        for o in Oav
            if o[2] == 1
                inf_start_time[o[1]] = min(inf_start_time[o[1]], o[3] - 5)
                maskauto[o[1]] = maskauto[o[1]] ∩ intervalUnion(o[3]-5, T)       
                θfrench[2,o[1]] = 10^10
                θfrench[3,o[1]] = o[3] - 5
                θfrench[4,o[1]] = 100 * T
            elseif o[2] == 0
                maskinf[o[1]] = maskinf[o[1]] ∩ intervalUnion(o[3], T)
                maskauto[o[1]] = maskauto[o[1]] ∩ intervalUnion(o[3], T)
                θfrench[1,o[1]] = 1e-10
            end
        end
        for i=1:N
            if inf_start_time[i] != T 
                maskauto[i] = maskauto[i] ∩ intervalUnion(inf_start_time[i], T)
            end
        end
        Mfrench = StochasticModel(HeuristicSI, T, θfrench, G, θp2gen, V);
        statsfre = prior(Mfrench,numsamples=2000);       
        p_cau = zeros(N,bins)
        p_french = zeros(N,bins)
        p_mh = zeros(N,bins)
        p_soft = zeros(N,bins)
        p_MF = MF_heu(N, T, G, Oav, 1/N, λ ; dt=1/4, maxit = 40)
        for i = 1:N
            for t = 1:bins
               p_cau[i,t] = marginal(i, t*T/bins, statscau)
               p_french[i,t] = marginal(i, t*T/bins, statsfre)
               p_mh[i,t] = marginal(i, t*T/bins, stats_mh)
               p_soft[i,t] = reweighted_marginal(i, t*T/bins, stats, weights_soft)
            end
        end
        AU_curve=zeros(bins)
        AU_sib=zeros(bins)
        AU_french = zeros(bins)
        AU_soft = zeros(bins)
        AU_MH = zeros(bins)
        AU_MF = zeros(bins)
        for t = 1:bins
            cau_risk=zeros(N)
            sib_risk = zeros(N)
            french_risk = zeros(N)
            MH_risk = zeros(N)
            MF_risk = zeros(N)
            soft_risk = zeros(N)
            for i=1:N
               cau_risk[i] = p_cau[i,t]
               french_risk[i] = p_french[i,t]
               MH_risk[i] = p_mh[i,t]
               MF_risk[i] = p_MF[i,Int(round(4*t))]
               sib_risk[i] = p_sib[i,Int(round(t/bins*nsteps_sib))]
               soft_risk[i] = p_soft[i,t]
            end
            xt = xtrue .< t * T / bins
            AU_soft[t] = AUROC(ROC(xt, soft_risk))
            AU_curve[t] = AUROC(ROC(xt, cau_risk))
            AU_french[t] = AUROC(ROC(xt, french_risk))
            AU_MH[t] = AUROC(ROC(xt, MH_risk))
            AU_MF[t] = AUROC(ROC(xt, MF_risk))
            AU_sib[t] = AUROC(ROC(xt, sib_risk))
        end
        open("$(folderstring)try$(trial)nobs$(nobs)cau.txt","w") do io
            writedlm(io,AU_curve') 
        end
        open("$(folderstring)try$(trial)nobs$(nobs)sib.txt","w") do io
            writedlm(io,AU_sib')  
        end
        open("$(folderstring)try$(trial)nobs$(nobs)fre.txt","w") do io
            writedlm(io,AU_french') 
        end
        open("$(folderstring)try$(trial)nobs$(nobs)mc.txt","w") do io
            writedlm(io,AU_MH') 
        end
        open("$(folderstring)try$(trial)nobs$(nobs)soft.txt","w") do io
            writedlm(io,AU_soft') 
        end
        open("$(folderstring)try$(trial)nobs$(nobs)mf.txt","w") do io
            writedlm(io,AU_MF') 
        end
        

    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:08:24
  F:  219.7599906741392
Progress: 100%|█████████████████████████████████████████| Time: 0:00:12
Progress:  24%|█████████▋                               |  ETA: 0:05:45